- 目前預設的firmware

In [ ]:
%sendtofile main.py

import machine, neopixel
import time

np = neopixel.NeoPixel(machine.Pin(12), 8)

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

for c in range(10):
    for i in range(8):
        np[i] = ((c*5*i)%50, (c*5*i*2)%50, (c*5*i*3)%50) # set to red, full brightness
    np.write()
    time.sleep(0.5)

for i in range(8):
    np[i] = (10, 10, 10) # set to red, full brightness
np.write()



## 加入MQTT的控制

In [ ]:
%websocketconnect --password 1234 ws://192.168.43.174:8266

In [ ]:
# send T/H sensor data to MQTT cloud platform
# control light with MQTT

from umqtt.simple import MQTTClient
from machine import Pin
import dht
import ubinascii
import machine
import network
import time
import os
import json

# ESP8266 ESP-12 modules have blue, active-low LED on GPIO2
led = Pin(2, Pin.OUT, value=1)
my_new_msg = None
TOPIC_BASE = 'pochang/iot'
topic_light = TOPIC_BASE+"/light"
topic_neopixel = TOPIC_BASE+"/neopixel"
topic_msg = TOPIC_BASE+'/msg'

#Control Function
def led_onoff(onoff):
    """ control led ON or OFF
        parameter:
        onoff
            0-->ON, 1-->OFF (acturely, led ON when level=0)
    """
    global led
    
    if(onoff==1):
        led.value(0)
    elif(onoff==-1):
        led.value(not led.value())
    else:
        led.value(1)

def neopixel_flash(neo_list=[[0,10,10]]*8, delay=0.5):
    np = neopixel.NeoPixel(machine.Pin(12), 8)

    for i in range(len(neo_list)):
        np[i] = neo_list[i]
    np.write()

    if delay > 0:
        time.sleep(delay)

        for i in range(8):
            np[i] = [0, 0, 0]
        np.write()
    
def sub_cb(topic, msg):
    global my_new_msg
    global TOPIC_BASE

    topic = topic.decode('utf-8')
    msg = msg.decode('utf-8')
    my_new_msg = '['+topic+'] '+ msg
    print(my_new_msg)
    print(msg[0], type(msg[0]), msg[0]=='{')
    
    if(topic == topic_light):
        if msg == "0":
            led_onoff(0)
        else:
            led_onoff(1)
    if(topic == topic_neopixel):
        if msg == "1":
            neopixel_flash()
        if msg[0]=='{': #json
            neo_cmd = json.loads(msg)
            neo_light = neo_cmd['light']
            neo_delay = neo_cmd['delay']
            neopixel_flash(neo_light, neo_delay)            
            

def main():
    global my_new_msg
    global TOPIC_BASE
    
    mq_fail_count = 0
    tm_pub_th = time.ticks_ms()

    led_onoff(1)

    #- check ap config file
    AP_SSID = 'upy'
    AP_PWD = 'pypypypy'
    ap_config = None
    ap_config_fn = 'ap.txt'
    if ap_config_fn in os.listdir():
        print('ap config here!')
        f = open(ap_config_fn)
        ap_config = f.read()
        f.close()
    if ap_config:
        print( ('ap_config:', ap_config))
        ap_config = ap_config.split('\n')
        AP_SSID = ap_config[0].strip()
        AP_PWD = ap_config[1].strip()
    print('link to: ', (AP_SSID, AP_PWD))

    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    wlan.connect(AP_SSID, AP_PWD)
    print('connecting to AP')
    while(not wlan.isconnected()):
        print(wlan.ifconfig())
        time.sleep(0.1)
        led_onoff(-1)
    print('connected!  --> ', wlan.ifconfig())
    
    # Default MQTT server to connect to
    #server = "iot.eclipse.org"
    server = "broker.hivemq.com"
    CLIENT_ID = ubinascii.hexlify(machine.unique_id()).decode('utf-8')
    
    c = MQTTClient(CLIENT_ID, server)
    # Subscribed messages will be delivered to this callback
    c.set_callback(sub_cb)
    c.connect()
    c.subscribe(topic_light)
    c.subscribe(topic_neopixel)
    print("Connected to %s, subscribed to %s topic" % (server, topic_light))

    # wifi ready, blink led
    for i in range(3):
        led_onoff(1)
        time.sleep(1)
        led_onoff(0)
        time.sleep(1)
    print('I am ready!, ID='+str(CLIENT_ID))
    c.publish(topic_msg, 'I am ready!, ID='+str(CLIENT_ID))

    try:
        while 1:
            if(not wlan.isconnected()):
                # not do any mq operation
                time.sleep(0.1)
                led_onoff(-1)                
                continue
            
            try:
                #c.wait_msg()
                c.check_msg()
                if my_new_msg:
                    c.publish(topic_msg, my_new_msg)
                    my_new_msg = None
                    
            except Exception as e:
                print('wlan:', wlan.isconnected())
                print('ex: ', str(e))
                mq_fail_count+=1
                time.sleep(1)
                
            try:
                if mq_fail_count>5:
                    mq_fail_count=0
                    c = MQTTClient(CLIENT_ID, server)
                    # Subscribed messages will be delivered to this callback
                    c.set_callback(sub_cb)
                    c.connect()
                    c.subscribe(topic_light)
                    c.subscribe(topic_neopixel)
                    print("Connected to %s, subscribed to %s topic" % (server, topic_light))
            except Exception as e:
                print('wlan:', wlan.isconnected())
                print('ex: ', str(e))
                    

            time.sleep(0.001)
                        
    finally:
        c.disconnect()


if __name__ == '__main__':
    main()


In [ ]:
import json

In [ ]:
jj = {'neopixel':[[10,10,0]]*8, 'delay':0.5}

In [ ]:
json_str = json.dumps(jj)

In [ ]:
neo_cmd = json.loads(json_str)
neo_light = neo_cmd['neopixel']
neo_delay = neo_cmd['delay']

neopixel_flash(neo_light, neo_delay)

In [ ]:
{"neopixel": [[10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0], [10, 0, 0]], "delay": 0.5}

## 如何修改要連線的AP

- 如果沒有經由WiFi AP連到MicroPython，而是直接連到MicroPython的WiFi (如：upy_01-02-03-04_192.168.43.11)，這時：
    - WiFi密碼預設為 12345678
    - MicroPython的IP為 192.168.4.1
    - 因此連線的magic command為 `%websocketconnect --password 1234 ws://192.168.4.1:8266`

In [ ]:
%websocketconnect --password 1234 ws://192.168.43.174:8266

In [ ]:
ap_config_fn = 'ap.txt'

In [ ]:
AP_SSID = 'malo-ap'
AP_PWD = '0928380233'
with open(ap_config_fn, 'w') as f:
    f.write(AP_SSID+'\n')
    f.write(AP_PWD+'\n')

In [ ]:
ap_config = None

print('ap config here!')
with open(ap_config_fn, 'r') as f:
    ap_config = f.read()

print(ap_config)
ap_config = ap_config.split('\n')
AP_SSID = ap_config[0].strip()
AP_PWD = ap_config[1].strip()
print(ap_config)